# Sonification of Bleeding with Bank of Filters (Nature)

# First Meeting (2019-06-11, Sasan and Thomas) @CITEC, TH proposed Filter-bank for feature generation
* The idea is to use a bank of different low-pass filters to create increasingly smooth signals
* these filtered signals serve as source for identifying key moments to anchor sound events
* which then create a multiscale data-driven complex grain structure of the raw instantaneous bleeding data.
* note that the limit of filtering with a cutoff-frequency towards 0 yields the integrated signal.

## Imports

In [ ]:
import os
import copy
import threading
import matplotlib
import ipywidgets
import scipy.interpolate
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from IPython.display import clear_output
from ipywidgets import interact, interactive, fixed, interact_manual, Layout


In [ ]:
#sonfn = [
#    "nature", "japan", "algomus", "eventearcons"]

#(son_nature_init, son_waterdrop_update, son_waterdrop_quit)
#sonfn = (son_waterdrop_init, son_waterdrop_update, son_waterdrop_quit)
#sonfn = (son_waterdrop_init, son_waterdrop_update, son_waterdrop_quit)

## Load Data and Create Filter-Bank Signals

In [ ]:
df = pd.read_csv('log_refactored_correction_factor.csv', na_values=['no info', '.'], delimiter=',')
df_indexed = df.reset_index(drop=False)
index = df_indexed['index']
delta = df_indexed['Delta']
volume = df_indexed['Blood Accumulated']

delta_min = delta.min()
delta_max = delta.max()

volume_min = volume.min()
volume_max = volume.max()

print("dataset loaded:")
print(f"  delta:   min={delta_min:8}, max={delta_max:8.3}")
print(f"  volume:  min={volume_min:8}, max={volume_max:8}")

## Event-based Sonification of filtered data (min/max/threshold cut-throughs...)

In [ ]:
import sc3nb as scn
import time
sc = scn.startup()

In [ ]:
#%sc FreqScope(400, 300)
#%sc s.makeGui
#%sc s.scope

## Load Buffers and SynthDef

In [ ]:
b = sc.Buffer().load_file("samples/birds.wav")
r = sc.Buffer().load_file("samples/rain.wav")
w = sc.Buffer().load_file("samples/water-flow.wav")
c = sc.Buffer().load_file("samples/crickets.wav")
s = sc.Buffer().load_file("samples/sheeps.wav")
t = sc.Buffer().load_file("samples/thunder.wav")
e = sc.Buffer().load_file("samples/seaguls.wav")
f = sc.Buffer().load_file("samples/footstep.wav")
k = sc.Buffer().load_file("samples/kalimba.wav")
m = sc.Buffer().load_file("samples/motor.wav")
l = sc.Buffer().load_file("samples/bell.wav")
h = sc.Buffer().load_file("samples/horse.wav")
i = sc.Buffer().load_file("samples/wind.wav")


In [ ]:
%%scv
SynthDef("pb-simple", { |out=0, bufnum=0, rate=1, pan=0, amp=0.3, loop=1, lgrt=2, lgamp = 0.5, cf=1000, rq=1, mix=0, room=0, damp=0.5|
    var sig;
    sig = PlayBuf.ar(2, bufnum, rate.lag(lgrt)*BufRateScale.kr(bufnum), loop: loop, doneAction: 2);
    sig = BPF.ar(sig, cf, rq);
    sig = FreeVerb2.ar(sig, sig, mix, room, damp);
    Out.ar(out, Pan2.ar(sig, pan, amp.lag(lgamp)));
}).add;


## Thread for Playing Data

In [ ]:
class Bloodplayer:
    
    def __init__(self, data, pulse_time=1, verbose=False):
        self.lock = threading.Lock()
        self.stopevent = threading.Event()
        self.callback_fn = None
        self.idx = 0
        self.data = data
        self.length = data.shape[0]
        self.verbose = verbose
        self.pulse_time = pulse_time
        
    #def __del__():
        # close plot window
        #pass
    
    def callback_fn_default(self, v):
        os.write(1, f"\r                       \r{v}".encode())
        
    def procfn(self):
        self.idx = 0
        while not self.stopevent.wait(0) and self.idx < self.length-1:
            v = self.data[self.idx]
            if self.verbose: 
                os.write(1, f"\r{self.idx}:{self.idx}                   ".encode())
            if callable(self.callback_fn):
                self.callback_fn(self)
            else:
                self.callback_fn_default(v)
            self.idx += 1
            time.sleep(self.pulse_time)
        print("done.")
    
    def set_callback(self, fn):
        self.callback_fn = fn
        
    def create_thread(self):
        threadname = "BloodPlayer-thread"
        # check first if it already exists
        if threadname in [t.name for t in threading.enumerate()]:
            print("create_thread: thread is already existing, stop first")
        else:
            self.stopevent.clear()
            self.producer = threading.Thread(name=threadname, target=self.procfn, args=[])
            self.producer.start()

    def stop_thread(self):
        self.stopevent.set()

In [ ]:
bloodplayer = Bloodplayer(delta)

## Plot Data

In [ ]:
%matplotlib

In [ ]:
# create figure
fig, ax = plt.subplots(1)  
mngr = plt.get_current_fig_manager(); 
mngr.window.setGeometry(840, 0, 600, 400)

# create axis
ax.clear()
plv0, = ax.plot([], [], "r-", lw=1.5)
plv1, = ax.plot([], [], "g-", lw=1.5)
plv2, = ax.plot([], [], "y-", lw=1.5)
plmarked, = ax.plot([], [], "r-", lw=1)
pldata, = ax.plot(delta, "-", lw=0.1)

def update_plot(self, t, x, v0, v1, v2, volume_val): 
    global fig, ax, plmarked, plv1, plv2, plv3
    
    plmarked.set_data([t,t], [-10, 10])
    plv0.set_xdata(x[0:])
    plv0.set_ydata(v0[0:])
    plv1.set_xdata(x[0:])
    plv1.set_ydata(v1[0:])
    plv2.set_xdata(x[0:])
    plv2.set_ydata(v2[0:])
    ax.draw_artist(ax.patch)
    ax.draw_artist(plmarked)
    ax.draw_artist(pldata)
    ax.draw_artist(plv0)
    ax.draw_artist(plv1)
    ax.draw_artist(plv2)
    progress.value = volume_val
    fig.canvas.update()

def onclick(event):
    global bloodplayer
    if event.dblclick:
        print(event.button, event.xdata)
        bloodplayer.idx = int(event.xdata)


connection_id = fig.canvas.mpl_connect('button_press_event', onclick)
      

## Sonification

In [ ]:
# Custom code for sonifications

v0 = [0,0]
v1 = [0,0]
v2 = [0,0]
vs0 = []
vs1 = []
vs2 = []
xs = []
tau = [5, 30, 120]
bn = (np.arange(5000, 5025, step=1, dtype=int)).astype(int).tolist()
takt = [0,0]
amp = [0,0,0]
rate = [0,0,0]
pan = [0,0,0]
takt_rate = 50
volume_val = 0
buf_nodes_one = [bn[0], bn[1], bn[3]]
bufnums_one = [w.bufnum, e.bufnum, m.bufnum]
buf_nodes_two = [bn[0], bn[4], bn[6]]    
bufnums_two = [w.bufnum, b.bufnum, h.bufnum]
buf_nodes_three = [bn[0], bn[4], bn[9]]    
bufnums_three = [w.bufnum, b.bufnum, i.bufnum]

# initiall play buffers for the selected sonification
def nature_init(buf_nodes, bufnums):
    sc.msg("/s_new", ["pb-simple", (buf_nodes[0]), 1, 1, "bufnum", bufnums[0], "rate", 1, "amp", 0])
    sc.msg("/s_new", ["pb-simple", (buf_nodes[1]), 1, 1, "bufnum", bufnums[1], "rate", 1, "amp", 0])
    sc.msg("/s_new", ["pb-simple", (buf_nodes[2]), 1, 1, "bufnum", bufnums[2], "rate", 1, "amp", 0])

def tau_zero(self, volume_val):
    # *** water ***
    # tau 0 = 5 seconds
    refidx = max(self.idx-tau[0], 0)
    v0[0] = (volume.values[self.idx] - volume.values[refidx])/tau[0]
    amp[0] = scn.linlin(v0[0], 0, 3, 0.2, 1)
    sc.msg("/n_set", [bn[0], "rate", 1, "amp", amp[0], "lgrt", 3, "lgamp", 1])
    if v0[0] >= 1.5:
        rate[0] = scn.linlin(v0[0], 1.5, 3.2, 1.5, 4)
        sc.msg("/n_set", [bn[0], "rate", rate[0]])

def event_off(buf_node):
    sc.msg("/s_new", ["pb-simple", buf_node, 1, 1, "bufnum", t.bufnum, "rate", 0.5, "amp", 0.4, "loop", 0, "lgrt", 2, "lgamp", 2, "cf", 400])
    time.sleep(2)
    sc.msg("/s_new", ["pb-simple", buf_node, 1, 1, "bufnum", t.bufnum, "rate", 0.3, "amp", 0.2, "loop", 0, "lgrt", 2, "lgamp", 3, "cf", 200])
    time.sleep(3)
    sc.msg("/s_new", ["pb-simple", buf_node, 1, 1, "bufnum", t.bufnum, "rate", 0.2, "amp", 0.05, "loop", 0, "lgrt", 1, "lgamp", 1, "cf", 100])
    time.sleep(1)
    sc.msg("/n_free", [buf_node])

def clock_event(takt):
    for t in range(1, takt+1): 
        sc.msg("/s_new", ["pb-simple", (bn[-1]+t), 1, 1, "bufnum", l.bufnum, "rate", 1, "amp", 0.1, "loop", 0])
        time.sleep(1)
    print("event clock: " + str(takt))

def plot_and_update(self, v0, v1, v2, volume_val):
    xs.append(self.idx)
    vs0.append(v0[0])
    vs1.append(v1[0])
    vs2.append(v2[0])
    update_plot(self, self.idx, xs, vs0, vs1, vs2, volume_val)
    
def nature_one_sonification(self):
    global tau, v0, v1, v2, vs0, vs1, vs2, xs, takt 
    global amp, rate, pan, takt_rate
    
    # assign volume
    volume_val = volume[self.idx]
    
    # *** water *** tau 0 = 5 seconds
    tau_zero(self, volume_val)
        
    # *** seaguls *** tau 1 = 30 seconds
    refidx = max(self.idx-tau[1], 0)
    v1[0] = (volume.values[self.idx] - volume.values[refidx])/tau[1]
    if v1[0] < 0.5:
        amp[1] = v1[0]/2
        sc.msg("/n_set", [bn[1], "rate", 1, "amp", amp[1]])

    if v1[0] >= 0.5:
        amp[1] = scn.linlin(v1[0], 0.5, 2, 0.25, 1)
        rate[1] = scn.linlin(v1[0], 0.5, 2, 1, 2)
        sc.msg("/n_set", [bn[1], "rate", rate[1], "amp", amp[1]])
        
        # *** thunder *** event for exceeding threshold! 
        if v1[1] < 0.7 and v1[0] >= 0.7:
            sc.msg("/s_new", ["pb-simple", bn[2], 1, 1, "bufnum", t.bufnum, "rate", 0.9, "amp", 0.9, "loop", 1, "lgrt", 2, "lgamp", 2, "cf", 500])
        if v1[0] < 0.7 and v1[1] >= 0.7:
            event_off(bn[2])
    
    v1[1] = v1[0]        

    # *** motor *** tau 2 = 2 minutes
    refidx = max(self.idx-tau[2], 0)
    v2[0] = (volume.values[self.idx] - volume.values[refidx])/tau[2]
    
    if v2[0] >= 0.3:
        amp[2] = scn.linlin(v2[0], 0.3, 0.5, 0.01, 0.07)
        rate[2] = scn.linlin(v2[0], 0.3, 0.5, 0.5, 0.8)
        pan[2] = scn.linlin(v2[0], 0.3, 0.5, -1, 0)
        sc.msg("/n_set", [bn[3], "rate", rate[2], "amp", amp[2], "pan", pan[2], "cf", 700, "mix", 0.2, "room", 0.2, "damp", 0.5])        

    os.write(1, f"\r{self.idx}, tau0: {float(v0[0]):4.2},  tau1: {float(v1[0]):4.2},  tau2: {float(v2[0]):4.2},   ".encode())

    # plot data
    plot_and_update(self, v0, v1, v2, volume_val)

    # clock-event for every 50 ml blood loss
    takt[0] = int(volume_val/takt_rate)
    if takt[0] > 0 and takt[0] != takt[1]:
        clock_event(takt[0])
    takt[1] = takt[0]

def nature_two_sonification(self):
    global tau, v0, v1, v2, vs0, vs1, vs2, xs, takt 
    global amp, rate, pan, takt_rate, volume_val
    
    # assign volume
    volume_val = volume[self.idx]
    
    # *** water *** tau 0 = 5 seconds
    tau_zero(self, volume_val)

    # *** birds *** tau 1 = 30 seconds
    refidx = max(self.idx-tau[1], 0)
    v1[0] = (volume.values[self.idx] - volume.values[refidx])/tau[1]
    if v1[0] < 0.5:
        amp[1] = v1[0]/2
        sc.msg("/n_set", [bn[4], "rate", 1, "amp", amp[1]])

    if v1[0] >= 0.5:
        amp[1] = scn.linlin(v1[0], 0.5, 2, 0.25, 1)
        rate[1] = scn.linlin(v1[0], 0.5, 2, 1, 2.5)
        sc.msg("/n_set", [bn[4], "rate", rate[1], "amp", amp[1]])
        
        # *** sheeps *** event for exceeding threshold!
        if v1[1] < 0.7 and v1[0] >= 0.7:
            sc.msg("/s_new", ["pb-simple", bn[5], 1, 1, "bufnum", s.bufnum, "rate", 1, "amp", 0.6, "loop", 1])
        if v1[0] < 0.7 and v1[1] >= 0.7:
            event_off(bn[5])

    v1[1] = v1[0]        

    # *** horse *** tau 2 = 2 minutes
    refidx = max(self.idx-tau[2], 0)
    v2[0] = (volume.values[self.idx] - volume.values[refidx])/tau[2]
    
    if v2[0] >= 0.25:
        amp[2] = scn.linlin(v2[0], 0.25, 0.5, 0.2, 0.8)
        rate[2] = scn.linlin(v2[0], 0.25, 0.5, 0.6, 1.2)
        pan[2] = scn.linlin(v2[0], 0.25, 0.5, 1, 0)

        sc.msg("/n_set", [bn[6], "rate", rate[2], "amp", amp[2], "pan", pan[2]])        

    os.write(1, f"\r{self.idx}, tau0: {float(v0[0]):4.2},  tau1: {float(v1[0]):4.2},  tau2: {float(v2[0]):4.2},   ".encode())
    
    # plot data
    plot_and_update(self, v0, v1, v2, volume_val)

    # clock-event for every 50 ml blood loss
    takt[0] = int(volume_val/takt_rate)
    if takt[0] > 0 and takt[0] != takt[1]:
        clock_event(takt[0])
    takt[1] = takt[0]

def nature_three_sonification(self):
    global tau, v0, v1, v2, vs0, vs1, vs2, xs, takt 
    global amp, rate, pan, takt_rate
    
    # assign volume
    volume_val = volume[self.idx]

    # *** water *** tau 0 = 5 seconds
    tau_zero(self, volume_val)

    # *** birds *** tau 1 = 30 seconds
    refidx = max(self.idx-tau[1], 0)
    v1[0] = (volume.values[self.idx] - volume.values[refidx])/tau[1]
    if v1[0] < 0.5:
        amp[1] = v1[0]/2
        sc.msg("/n_set", [bn[4], "rate", 1, "amp", amp[1]])

    if v1[0] >= 0.5:
        amp[1] = scn.linlin(v1[0], 0.5, 2, 0.25, 1)
        sc.msg("/n_set", [bn[4], "rate", 1, "amp", amp[1]])

        # *** thunder *** event for exceeding threshold!
        if v1[1] < 0.7 and v1[0] >= 0.7:
            sc.msg("/s_new", ["pb-simple", bn[2], 1, 1, "bufnum", t.bufnum, "rate", 1, "amp", 1, "loop", 1])
        if v1[0] < 0.7:
            sc.msg("/n_free", [bn[2]])
    
    v1[1] = v1[0]        

    # *** wind *** tau 2 = 2 minutes
    refidx = max(self.idx-tau[2], 0)
    v2[0] = (volume.values[self.idx] - volume.values[refidx])/tau[2]
    
    if v2[0] >= 0.25:
        amp[2] = scn.linlin(v2[0], 0.25, 0.5, 0.01, 0.1)
        rate[2] = scn.linlin(v2[0], 0.25, 0.5, 0.6, 1)
        pan[2] = scn.linlin(v2[0], 0.25, 0.5, 1, 0)

        sc.msg("/n_set", [bn[9], "rate", rate[2], "amp", amp[2], "pan", pan[2]])        

    os.write(1, f"\r{self.idx}, tau0: {float(v0[0]):4.2},  tau1: {float(v1[0]):4.2},  tau2: {float(v2[0]):4.2},   ".encode())
    
    # plot data
    plot_and_update(self, v0, v1, v2, volume_val)
    
    # clock-event for every 50 ml blood loss
    takt[0] = int(volume_val/takt_rate)
    if takt[0] > 0 and takt[0] != takt[1]:
        clock_event(takt[0])
    takt[1] = takt[0]

def quit():
    sc.free_all()


## GUI

In [ ]:
# GUI

selected_init = 0

# filling blood bar
progress = ipywidgets.FloatProgress(value=5, min=0, max=volume_max, description="Blood Loss", bar_style='danger')
progress.orientation = 'vertical'
display(progress)

def select_sonification(selection):
    global selected_init
    bloodplayer.set_callback(selection)
    if selection == nature_one_sonification:
        selected_init = 1
    if selection == nature_two_sonification:
        selected_init = 2
    if selection == nature_three_sonification:
        selected_init = 3

interact(select_sonification, selection = {
    'nature_one': nature_one_sonification,
    'nature_two': nature_two_sonification,
    'nature_three': nature_three_sonification
})

def start(b):
    global bloodplayer
    bloodplayer.create_thread()
    if selected_init == 1:
        nature_init(buf_nodes_one, bufnums_one)
        os.write(2, f"          one initiated          ".encode())
    if selected_init == 2:
        nature_init(buf_nodes_two, bufnums_two)
        os.write(2, f"          two initiated          ".encode())
    if selected_init == 3:
        nature_init(buf_nodes_three, bufnums_three)
        os.write(2, f"          three initiated          ".encode())
    print("start")
    
b1 = ipywidgets.Button(description='Start') 
b1.on_click(start)

def stop(b):
    global bloodplayer
    print("stop")
    bloodplayer.stop_thread()
    quit()
    
b2 = ipywidgets.Button(description='Stop') 
b2.on_click(stop)

out = ipywidgets.Output()
ipywidgets.HBox([b1, b2, out])

